# Прогнозирование температуры плавления стали

**Описание проекта**

Чтобы оптимизировать производственные расходы, металлургический комбинат ООО «Так закаляем сталь» решил уменьшить потребление электроэнергии на этапе обработки стали. Вам предстоит построить модель, которая предскажет температуру стали.

**Описание этапа обработки стали**

Сталь обрабатывают в металлическом ковше вместимостью около 100 тонн. Чтобы ковш выдерживал высокие температуры, изнутри его облицовывают огнеупорным кирпичом. Расплавленную сталь заливают в ковш и подогревают до нужной температуры графитовыми электродами. Они установлены в крышке ковша.

Из сплава выводится сера (десульфурация), добавлением примесей корректируется химический состав и отбираются пробы. Сталь легируют — изменяют её состав — подавая куски сплава из бункера для сыпучих материалов или проволоку через специальный трайб-аппарат (англ. tribe, «масса»).

Перед тем как первый раз ввести легирующие добавки, измеряют температуру стали и производят её химический анализ. Потом температуру на несколько минут повышают, добавляют легирующие материалы и продувают сплав инертным газом. Затем его перемешивают и снова проводят измерения. Такой цикл повторяется до достижения целевого химического состава и оптимальной температуры плавки.

Тогда расплавленная сталь отправляется на доводку металла или поступает в машину непрерывной разливки. Оттуда готовый продукт выходит в виде заготовок-слябов (англ. slab, «плита»).

**Описание данных**

Данные состоят из файлов, полученных из разных источников:

1. `data_arc.csv` — данные об электродах;
2. `data_bulk.csv` — данные о подаче сыпучих материалов (объём);
3. `data_bulk_time.csv` — данные о подаче сыпучих материалов (время);
4. `data_gas.csv` — данные о продувке сплава газом;
5. `data_temp.csv` — результаты измерения температуры;
6. `data_wire.csv` — данные о проволочных материалах (объём);
7. `data_wire_time.csv` — данные о проволочных материалах (время).

Во всех файлах столбец `key` содержит номер партии. В файлах может быть несколько строк с одинаковым значением `key`: они соответствуют разным итерациям обработки.

**План работы**

1. **Предобработка и исследовательский анализ данных**. Проверить наличие пропусков, некорректных значений, дубликатов, провести их обработку, привести данные к нужному типу. Изучить признаки, их основные статистики, построить графики при необходимости. Удалить признаки, не несущие смысловой нагрузки или явлющиеся утечкой таргета. Подготовить итоговую таблицу "объектов-признаков", где объектами являются уникальные ключи партий.
   
2. **Обучение моделей**. Разделить выборку на обучающую и тестовую. Исследовать разные ML модели. Выбрать признаковое пространство, модель и ее гиперпараметры, имеющее лучшее качество на кросс-валидации
   
3. **Оценка эффективности моделей на тестовой выборке**. При недостижении целевого значения метрики качества вернуться к этапу 2, сформировать дополнительные признаки, продолжить выбор моделей и настройку гиперпараметров.

In [1]:
import pandas as pd
import numpy as np
import optuna
import plotly.express as px

from caseconverter import snakecase
from collections import defaultdict
from IPython.display import display

from ydata_profiling import ProfileReport
from fast_ml import eda

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from category_encoders import MEstimateEncoder

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error

In [2]:
FIG_WIDTH = 9 * 100
FIG_HEIGHT = 5 * 100
RANDOM_SEED = 42

In [14]:
try:
    raw = {
        'arc': pd.read_csv('data_arc.csv'),
        'bulk_t': pd.read_csv('data_bulk_time.csv'),
        'bulk': pd.read_csv('data_bulk.csv'),
        'gas': pd.read_csv('data_gas.csv'),
        'temp': pd.read_csv('data_temp.csv'),
        'wire_t': pd.read_csv('data_wire_time.csv'),
        'wire': pd.read_csv('data_wire.csv'),
    }
except:
    raw = {
        'arc': pd.read_csv('/datasets/data_arc.csv'),
        'bulk_t': pd.read_csv('/datasets/data_bulk_time.csv'),
        'bulk': pd.read_csv('/datasets/data_bulk.csv'),
        'gas': pd.read_csv('/datasets/data_gas.csv'),
        'temp': pd.read_csv('/datasets/data_temp.csv'),
        'wire_t': pd.read_csv('/datasets/data_wire_time.csv'),
        'wire': pd.read_csv('/datasets/data_wire.csv'),
    }

## Исследовательский анализ данных

Изучим основные зависимости в данных перед тем, как мы будем использовать их в алгоритмах машинного обучения.

Таблица-резюме:

In [24]:
for dataset in raw.keys():
    print('#' * 50, 'Summary for', dataset, 'dataset', '#' * 50)
    display(eda.df_info(raw[dataset]))

################################################## Summary for arc dataset ##################################################


,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
key,int64,Numerical,3214,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
Начало нагрева дугой,object,Categorical,14875,"[2019-05-03 11:02:14, 2019-05-03 11:07:28, 201...",0,0.0
Конец нагрева дугой,object,Categorical,14876,"[2019-05-03 11:06:02, 2019-05-03 11:10:33, 201...",0,0.0
Активная мощность,float64,Numerical,14869,"[0.9760587506, 0.8056070805, 0.7443634272, 1.6...",0,0.0
Реактивная мощность,float64,Numerical,14866,"[0.6870837212, 0.5202852488, 0.4988054037, 1.0...",0,0.0


################################################## Summary for bulk_t dataset ##################################################


,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
key,int64,Numerical,3129,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
Bulk 1,object,Categorical,252,"[nan, 2019-05-03 17:50:19, 2019-05-05 10:09:13...",2877,91.946309
Bulk 2,object,Categorical,22,"[nan, 2019-05-07 08:36:19, 2019-05-07 10:05:35...",3107,99.2969
Bulk 3,object,Categorical,1298,"[nan, 2019-05-03 20:13:36, 2019-05-04 03:47:10...",1831,58.517098
Bulk 4,object,Categorical,1014,"[2019-05-03 11:21:30, 2019-05-03 11:46:38, 201...",2115,67.59348
Bulk 5,object,Categorical,77,"[nan, 2019-05-07 08:36:19, 2019-05-07 10:05:35...",3052,97.53915
Bulk 6,object,Categorical,576,"[nan, 2019-05-03 18:52:57, 2019-05-03 21:15:20...",2553,81.591563
Bulk 7,object,Categorical,25,"[nan, 2019-05-07 11:37:42, 2019-05-15 09:35:53...",3104,99.201023
Bulk 8,object,Categorical,1,"[nan, 2019-07-05 17:46:11]",3128,99.968041
Bulk 9,object,Categorical,19,"[nan, 2019-05-14 00:04:56, 2019-05-14 00:37:47...",3110,99.392777


################################################## Summary for bulk dataset ##################################################


...

,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
key,int64,Numerical,3216,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
Время замера,object,Categorical,15907,"[2019-05-03 11:16:18, 2019-05-03 11:25:53, 201...",0,0.0
Температура,float64,Numerical,175,"[1571.0, 1604.0, 1618.0, 1601.0, 1613.0, 1581....",2901,18.237254


################################################## Summary for wire_t dataset ##################################################


,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
key,int64,Numerical,3081,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
Wire 1,object,Categorical,3055,"[2019-05-03 11:11:41, 2019-05-03 11:46:10, 201...",26,0.843882
Wire 2,object,Categorical,1079,"[nan, 2019-05-03 13:32:06, 2019-05-03 13:59:54...",2002,64.978903
Wire 3,object,Categorical,63,"[nan, 2019-05-04 04:47:53, 2019-05-04 05:40:08...",3018,97.955209
Wire 4,object,Categorical,14,"[nan, 2019-05-07 08:48:23, 2019-05-07 10:14:55...",3067,99.545602
Wire 5,object,Categorical,1,"[nan, 2019-08-08 16:01:07]",3080,99.967543
Wire 6,object,Categorical,73,"[nan, 2019-05-07 08:37:20, 2019-05-07 10:06:35...",3008,97.630639
Wire 7,object,Categorical,11,"[nan, 2019-07-23 12:42:00, 2019-07-23 12:08:44...",3070,99.642973
Wire 8,object,Categorical,19,"[nan, 2019-05-14 00:03:32, 2019-05-14 00:36:39...",3062,99.383317
Wire 9,object,Categorical,29,"[nan, 2019-05-04 16:09:11, 2019-06-01 02:25:26...",3052,99.058747


################################################## Summary for wire dataset ##################################################


,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
key,int64,Numerical,3081,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
Wire 1,float64,Numerical,2251,"[60.059998, 96.052315, 91.160157, 89.063515, 8...",26,0.843882
Wire 2,float64,Numerical,713,"[nan, 9.11456, 9.143681, 12.376, 52.416003, 37...",2002,64.978903
Wire 3,float64,Numerical,56,"[nan, 250.041794, 276.324066, 96.047951, 88.64...",3018,97.955209
Wire 4,float64,Numerical,14,"[nan, 24.148801, 43.065361, 33.182243, 103.034...",3067,99.545602
Wire 5,float64,Numerical,1,"[nan, 15.132]",3080,99.967543
Wire 6,float64,Numerical,69,"[nan, 60.094322000000005, 54.122643, 64.212723...",3008,97.630639
Wire 7,float64,Numerical,10,"[nan, 5.035472, 8.49004, 1.053936, 0.234208, 1...",3070,99.642973
Wire 8,float64,Numerical,13,"[nan, 46.002319, 46.094879, 46.187439000000005...",3062,99.383317
Wire 9,float64,Numerical,25,"[nan, 35.0532, 4.622800000000001, 35.0168, 10....",3052,99.058747


Числовые распределения:

In [41]:
for dataset in raw.keys():
    print('#' * 50, 'Summary for', dataset, 'dataset', '#' * 50)
    display(round(raw[dataset].describe(include='all').T, 2))

################################################## Summary for arc dataset ##################################################


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
key,14876.0,NaN,NaN,NaN,1615.220422,934.571502,1.0,806.0,1617.0,2429.0,3241.0
Начало нагрева дугой,14876,14875,2019-06-10 22:02:03,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Конец нагрева дугой,14876,14876,2019-05-03 11:06:02,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Активная мощность,14876.0,NaN,NaN,NaN,0.670441,0.408159,0.030002,0.395297,0.555517,0.857034,3.731596
Реактивная мощность,14876.0,NaN,NaN,NaN,0.452592,5.878702,-715.504924,0.290991,0.415962,0.637371,2.676388


################################################## Summary for bulk_t dataset ##################################################


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
key,3129.0,NaN,NaN,NaN,1624.383509,933.337642,1.0,816.0,1622.0,2431.0,3241.0
Bulk 1,252,252,2019-05-03 17:50:19,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bulk 2,22,22,2019-05-07 08:36:19,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bulk 3,1298,1298,2019-05-03 20:13:36,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bulk 4,1014,1014,2019-05-03 11:21:30,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bulk 5,77,77,2019-05-07 08:36:19,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bulk 6,576,576,2019-05-03 18:52:57,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bulk 7,25,25,2019-05-07 11:37:42,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bulk 8,1,1,2019-07-05 17:46:11,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bulk 9,19,19,2019-05-14 00:04:56,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


################################################## Summary for bulk dataset ##################################################


...

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
key,15907.0,NaN,NaN,NaN,1607.88087,942.212073,1.0,790.0,1618.0,2427.0,3241.0
Время замера,15907,15907,2019-05-03 11:16:18,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Температура,13006.0,NaN,NaN,NaN,1591.84092,21.375851,1191.0,1581.0,1591.0,1601.0,1705.0


################################################## Summary for wire_t dataset ##################################################


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
key,3081.0,NaN,NaN,NaN,1623.426485,932.996726,1.0,823.0,1619.0,2434.0,3241.0
Wire 1,3055,3055,2019-05-03 11:11:41,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wire 2,1079,1079,2019-05-03 13:32:06,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wire 3,63,63,2019-05-04 04:47:53,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wire 4,14,14,2019-05-07 08:48:23,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wire 5,1,1,2019-08-08 16:01:07,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wire 6,73,73,2019-05-07 08:37:20,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wire 7,11,11,2019-07-23 12:42:00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wire 8,19,19,2019-05-14 00:03:32,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wire 9,29,29,2019-05-04 16:09:11,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


################################################## Summary for wire dataset ##################################################


,count,mean,std,min,25%,50%,75%,max
key,3081.0,1623.43,933.00,1.00,823.00,1619.00,2434.00,3241.00
Wire 1,3055.0,100.90,42.01,1.92,72.12,100.16,126.06,330.31
Wire 2,1079.0,50.58,39.32,0.03,20.19,40.14,70.23,282.78
Wire 3,63.0,189.48,99.51,0.14,95.14,235.19,276.25,385.01
Wire 4,14.0,57.44,28.82,24.15,40.81,45.23,76.12,113.23
Wire 5,1.0,15.13,NaN,15.13,15.13,15.13,15.13,15.13
Wire 6,73.0,48.02,33.92,0.03,25.05,42.08,64.21,180.45
Wire 7,11.0,10.04,8.61,0.23,6.76,9.02,11.89,32.85
Wire 8,19.0,53.63,16.88,45.08,46.09,46.28,48.09,102.76
Wire 9,29.0,34.16,19.93,4.62,22.06,30.07,43.86,90.05


И детальный отчет:

In [42]:
for dataset in raw.keys():
    print('#' * 50, 'Summary for', dataset, 'dataset', '#' * 50)
    ProfileReport(raw[dataset]).to_widgets()

################################################## Summary for arc dataset ##################################################


Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

################################################## Summary for bulk_t dataset ##################################################


Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

################################################## Summary for bulk dataset ##################################################


...

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

################################################## Summary for wire_t dataset ##################################################


Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

################################################## Summary for wire dataset ##################################################


Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

**Выводы на основе предварительного анализа:**

Данные о каждой партии разбиты по датасетам. Всего собрана информация о 3239 партий. Названия столбцов встречаются на русском и английском языках. Их надо будет привести к формату `snake_case`. Некоторым столбцам назначен неправильный тип данных. Во всех датасетах присутствует столбец `key`, содержащий номер партии.

1. В датасете `data_arc_new.csv` 14876 записей о нагреве сплава. Явное дублирование и пропуски - отсутствуют. Столбцам `Начало нагрева дугой` и `Конец нагрева дугой` необходимо сменить тип данных на `datetime`. Название столбцов требуется перевести на английский язык.
   
2. В датасете `data_bulk_new.csv` 3129 сплава и записи о добавленных в них сыпучих присадок. Всего существует 15 присадок. Явное дублирование - отсутствует. Пропуски означают, что данная присадка не добавлялась в сплав. Пропуски будут заменены значением `0.0`.
   
3. В датасете `data_bulk_time_new.csv` аналогичные с предыдущим пунктом замечания. Дополнительно необходимо сменить тип данных на `datetime`.
   
4. В датасете `data_gas_new.csv` 3239 записей о подаче газа в сплавы. Явное дублирование и пропуски - отсутствуют. Название столбца `Газ 1` требуется перевести на английский язык.
   
5. В датасете `data_temp_new.csv` 18092 замеров температур. Явное дублирование - отсутствует. В столбце `Температура` встречаются пропуски. К сожалению, эти строки придется удалить, т.к. нет возможности восстановить пропущенные значения. Для столбца `Время замера` необходимо сменить тип данных на `datetime`. Название столбцов требуется перевести на английский язык.
   
6. Датасеты `data_wire_new.csv` и `data_wire_time_new.csv` содержат 3081 сплав и записи о добавленных в них проволочных присадок. Остальные замечания аналогичны как для датасетов `data_bulk_new.csv` и `data_bulk_time_new.csv`.

## Предобработка данных

Данные уже достаточно чистые, поэтому обработка будет относительно быстрой:

1. Создадим новые колонки с признаками.
2. Обрежем нулевые значения.